In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


# 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
# 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").

# 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
# 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "question": "Top Gun",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "question": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝"
    },
    {
        "question": "Titanic",
        "answer": "🛳️💕🧊"
    },

]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you think about {question}? Please give me three emojis that represent the movie."),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you reply with three emojis that represent the movie. But if the user's question is a general type of question, not a movie title, please provide an answer that is appropriate to the question."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


invoke_chain("matrix")

invoke_chain("3 idiots")



content='🕶️💊🤖'
content='🎓🤪📚'


In [14]:
invoke_chain("What is the movie name which I asked lately")


content='The movie you asked about is "3 Idiots."'
